# DMS-MaPseq
**Note: first run of this cell takes a while**


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../src')
from util import *
from config import *
import plots
import ipynbname
from study_gen import study

import plotly.graph_objects as go
from plotly.subplots import make_subplots

### # Mutations per read
- histogram
- x-axis = # mutations
- y-axis = # reads


In [ ]:

unique_samples = study.df['sample'].unique()
fig = make_subplots(rows=len(unique_samples), cols=1, vertical_spacing=0.4/len(unique_samples),
                     subplot_titles=['Number of mutations per read - {}'.format(sample) for sample in unique_samples])
for i_s, sample in enumerate(unique_samples):
    fig.add_trace( plots.mutations_per_read(study, sample), row=i_s+1, col=1 )
    fig.update_yaxes(title='Count')
    fig.update_xaxes(dtick=10)


fig.update_layout(autosize=True, height=10000, title='Number of mutation per read across samples')

save_plotly_fig(ipynbname.path(), '[A] Mutations per read', fig)
plot = {
    'fig':fig,
    'data':study.df[study.df['section']=='full'][['sample','construct','num_of_mutations']]
    }
plot['fig'].show()


### Mutation identity at each position
- stacked bar graph (ACUG coloring)
- x-axis = position (number/base/both??)
- y-axis = # reads


In [ ]:
## Change sample and family ##
sample = '01_02_S23_reads'   
family = 'hp1'    
##############################

unique_constructs = study.df[(study.df['sample']==sample)&(study.df['family']==family)]['construct'].unique()

fig = make_subplots(rows=len(unique_constructs), cols=1, vertical_spacing=0.2/len(unique_constructs),
                    subplot_titles=['Mutation identity at each position - {}'.format(cst) for cst in unique_constructs])
for i_c, construct in enumerate(unique_constructs):
    muts_identity = plots.mutation_identity_at_each_position(study, sample, construct)

    for bar in muts_identity['fig']:
        fig.add_trace( bar, row=i_c+1, col=1 )
    
    fig.update_xaxes(tickangle=0, 
            tickvals=np.arange(len(muts_identity['data'].index)), ticktext=list(muts_identity['data'].index), tickfont={'size':8},
            row=i_c+1, col=1)
        
for trace, name in zip(fig["data"][:4], ['A','C','G','T']):
    trace.update(showlegend=True)
    trace["name"] = name

fig.update_yaxes(title='Mutation fraction')
fig.update_layout(barmode='stack', height=500*len(unique_constructs), width=1500)
save_plotly_fig(ipynbname.path(), '[B] Mutation identity at each position/{}/{}'.format(sample, family), fig)
plot = {
    'fig':fig,
    'data':study.df[
        (study.df['sample']==sample)&
        (study.df['family']==family)&
        (study.df['section']=='full')]\
    [['sample','construct','mod_bases_A','mod_bases_C','mod_bases_G','mod_bases_T','num_aligned']]
    }
plot['fig'].show()

### Mutation fraction at each position 
- bar graph (ACUG coloring)
- x-axis = position (number/base/both??)
- y-axis = # reads


In [19]:
## Change sample and family ##
sample = '01_02_S23_reads'   
family = 'hp1'    
##############################    

unique_constructs = study.df[(study.df['sample']==sample)&(study.df['family']==family)]['construct'].unique()

fig = make_subplots(rows=len(unique_constructs), cols=1, vertical_spacing=0.2/len(unique_constructs),
                    subplot_titles=['Mutation fraction at each position - {}'.format(cst) for cst in unique_constructs])
for i_c, construct in enumerate(unique_constructs):
    muts_identity = plots.mutation_fraction_at_each_position(study, sample, construct)

    for bar in muts_identity['fig']:
        fig.add_trace( bar, row=i_c+1, col=1 )
    
    fig.update_xaxes(tickangle=0, 
            tickvals=np.arange(len(muts_identity['data'].index)), ticktext=list(muts_identity['data'].index), tickfont={'size':8},
            row=i_c+1, col=1)
        
for trace, name in zip(fig["data"][:4], ['A','C','G','T']):
    trace.update(showlegend=True)
    trace["name"] = name

fig.update_yaxes(title='Mutation fraction')
fig.update_layout(barmode='stack', height=500*len(unique_constructs), width=1500)
save_plotly_fig(ipynbname.path(), '[C] Mutation fraction at each position/{}/{}'.format(sample, family), fig)
plot = {
    'fig':fig,
    'data':study.df[
        (study.df['sample']==sample)&
        (study.df['family']==family)&
        (study.df['section']=='full')]\
    [['sample','construct','mut_rates','num_aligned']]
    }
plot['fig'].show()


,sample,construct,mut_rates,num_aligned
119366,01_02_S23_reads,3042-O-flank_1=hp1-DB,"[0.0, 0.0, 0.00011253657438667567, 0.001750437...",10082
119374,01_02_S23_reads,3043-CC-flank_1=hp1-DB,"[0.0, 0.0, 0.0002487098178200584, 0.0018401144...",18398
119382,01_02_S23_reads,3044-O-flank_1=hp1-DB,"[0.0, 0.0, 0.00011795234725171031, 0.002197234...",9725
119390,01_02_S23_reads,3045-CC-flank_1=hp1-DB,"[0.0, 0.0, 0.0, 0.0016827462418667265, 0.00080...",11174
119398,01_02_S23_reads,3046-O-flank_1=hp1-DB,"[0.0, 0.0, 0.00011275228323373548, 0.002165329...",10010
119406,01_02_S23_reads,3047-O-flank_1=hp1-DB,"[0.0, 0.0, 8.444519506840061e-05, 0.0037993180...",13583
119414,01_02_S23_reads,3048-O-flank_1=hp1-DB,"[0.0, 0.0, 0.00013175230566534913, 0.003518971...",8554
119422,01_02_S23_reads,3049-O-flank_1=hp1-DB,"[0.0, 0.0, 0.0006108735491753207, 0.0028328611...",1869
119430,01_02_S23_reads,3050-O-flank_1=hp1-DB,"[0.0, 0.0, 0.0005106600280863015, 0.0035765379...",8983
119438,01_02_S23_reads,3051-O-flank_1=hp1-DB,"[0.0, 0.0, 0.0, 0.007348560079443893, 0.000663...",13852


### Read coverage per position
- bar graph
- x-axis = position (number/base/both??)
- y-axis = coverage fraction

In [33]:
## Edit sample and family here ##
sample = '01_02_S23_reads'   
family = 'hp1'    
#################################   

unique_constructs = study.df[(study.df['sample']==sample)&(study.df['family']==family)]['construct'].unique()

fig = make_subplots(rows=len(unique_constructs), cols=1, vertical_spacing=0.2/len(unique_constructs),
                    subplot_titles=['Read coverage per position - {}'.format(cst) for cst in unique_constructs])
for i_c, construct in enumerate(unique_constructs):
    read_coverage = plots.read_coverage_per_position(study, sample, construct)

    for bar in read_coverage['fig']:
        fig.add_trace( bar, row=i_c+1, col=1 )

# print a legend for each section
for trace, name in zip(fig["data"][:len(read_coverage['data']['section'])], read_coverage['data']['section']):
    trace.update(showlegend=True)
    trace["name"] = name

fig.update_yaxes(title='Read coverage')
fig.update_layout(barmode='stack', height=500*len(unique_constructs), width=1300)
save_plotly_fig(ipynbname.path(), '[D] Read coverage per position/{}/{}'.format(sample, family), fig)
plot = {
    'fig':fig,
    'data':study.df[
        (study.df['sample']==sample)&
        (study.df['family']==family)&
        (study.df['section']=='full')]\
    [['sample','construct','cov_bases']]
    }
#plot['fig'].show()

plot['data']

,sample,construct,cov_bases
119366,01_02_S23_reads,3042-O-flank_1=hp1-DB,"[8926, 9308, 9357, 9748, 9841, 9907, 9981, 100..."
119374,01_02_S23_reads,3043-CC-flank_1=hp1-DB,"[16313, 16971, 17061, 17771, 17927, 18050, 182..."
119382,01_02_S23_reads,3044-O-flank_1=hp1-DB,"[8611, 8981, 9021, 9385, 9475, 9546, 9632, 968..."
119390,01_02_S23_reads,3045-CC-flank_1=hp1-DB,"[9914, 10315, 10362, 10831, 10930, 10998, 1107..."
119398,01_02_S23_reads,3046-O-flank_1=hp1-DB,"[8963, 9309, 9373, 9745, 9819, 9873, 9948, 998..."
119406,01_02_S23_reads,3047-O-flank_1=hp1-DB,"[12033, 12538, 12615, 13149, 13255, 13336, 134..."
119414,01_02_S23_reads,3048-O-flank_1=hp1-DB,"[7667, 7975, 8016, 8303, 8371, 8430, 8489, 851..."
119422,01_02_S23_reads,3049-O-flank_1=hp1-DB,"[1637, 1716, 1725, 1801, 1816, 1823, 1838, 185..."
119430,01_02_S23_reads,3050-O-flank_1=hp1-DB,"[7971, 8288, 8334, 8697, 8765, 8820, 8913, 894..."
119438,01_02_S23_reads,3051-O-flank_1=hp1-DB,"[12191, 12691, 12757, 13333, 13482, 13589, 137..."
